In [18]:
 !pip install gym


In [2]:
import pandas as pd
import gym
from gym import spaces
import numpy as np
ratings_df = pd.read_csv('ratings.csv')
movies_df = pd.read_csv('movies.csv')




In [3]:
movies_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
10324,146684,Cosmic Scrat-tastrophe (2015),Animation|Children|Comedy
10325,146878,Le Grand Restaurant (1966),Comedy
10326,148238,A Very Murray Christmas (2015),Comedy
10327,148626,The Big Short (2015),Drama


In [4]:
ratings_df

,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523
...,...,...,...,...
105334,668,142488,4.0,1451535844
105335,668,142507,3.5,1451535889
105336,668,143385,4.0,1446388585
105337,668,144976,2.5,1448656898


In [5]:


class MovieLensEnv(gym.Env):
    """
     Environnement personnalisé qui utilise gym interface et simule
    un système de recommandation basé sur l'ensemble de données MovieLens.
    """
    metadata = {'render.modes': ['console']}
    
    def __init__(self, user_profiles, movie_list):
        super(MovieLensEnv, self).__init__()
        # Définir les espaces d'action et d'observation
        # Ils doivent être des objets gym.spaces
        # Exemple d'utilisation d'actions discrètes, nous avons une action pour chaque film
        self.action_space = spaces.Discrete(len(movie_list))
        # L'observation sera le profil de l'utilisateur, qui pourrait inclure
        # des données démographiques sur l'utilisateur et des évaluations de films
        self.observation_space = spaces.Box(low=0, high=5, shape=(len(user_profiles[0]),), dtype=np.float32)
        
        self.user_profiles = user_profiles
        self.movie_list = movie_list
        self.current_user = 0  # Garder trace de l'index de l'utilisateur actuel

    def step(self, action):
        # 'action' est le movie ID du film à recommander
        current_profile = self.user_profiles[self.current_user]

        #Trouver l'index dans 'self.movie_list' qui correspond à movie ID dans 'action'
        try:
            action_index = np.where(self.movie_list == action)[0][0]
        except IndexError:
            raise ValueError(f"Movie ID {action} is not found in the movie list.")
    
        # Simuler l'action de notation de l'utilisateur à l'aide de l'index
        reward, done = self.simulate_user_rating(current_profile, action_index)
        self.current_user = (self.current_user + 1) % len(self.user_profiles)
        next_state = self.user_profiles[self.current_user]

        return next_state, reward, done, {}

    
    def reset(self):
         #Réinitialiser l'état de l'environnement à un état initial
        self.current_user = 0
        return self.user_profiles[self.current_user]  # renvoie le premier profil d'utilisateur
    
    def render(self, mode='console'):
        if mode != 'console':
            raise NotImplementedError()
        # Pour simplifier,on va juste afficher le profil et l'action de l'utilisateur actuel.
        print(f"User Profile: {self.user_profiles[self.current_user]}")
    
    def simulate_user_rating(self, user_profile, recommended_movie_id):
        # Vérifie si l'utilisateur a noté le film
        user_ratings = ratings_df[ratings_df['userId'] == self.current_user]
        if recommended_movie_id in user_ratings['movieId'].values:
            # Si l'utilisateur a noté le film, on utilise cette note comme recompense
            reward = user_ratings[user_ratings['movieId'] == recommended_movie_id]['rating'].iloc[0]
            done = True  
        else:
            # si l'utilisateur n'a pas noté le film, on lui donne une recompense neutre qui peut etre la note moyenne de l'utilisateur
           
            reward = user_ratings['rating'].mean() if not user_ratings.empty else 2.5
            done = False  
            
        # Ici, nous pourrions également mettre en œuvre un mécanisme pour estimer la note si le film n'a pas été noté
        # Par exemple, en utilisant un modèle de filtrage collaboratif ou basé sur le contenu pour prédire la note.
        
        return reward, done


In [6]:

user_id = 1  #on peut le remplacer par n'importe quel autre user_Id
# creer une liste de tous les movie IDs
all_movie_ids = movies_df['movieId'].unique()
#Initialiser le profil de l'utilisateur avec des zéros pour chaque film
user_profile = np.zeros(len(all_movie_ids))

# Obtenir toutes les notes de l'utilisateur
user_ratings = ratings_df[ratings_df['userId'] == user_id]

# Mise à jour du profil de l'utilisateur avec les notes de l'utilisateur
for movie_id, rating in user_ratings[['movieId', 'rating']].values:
    movie_index = np.where(all_movie_ids == movie_id)[0][0]  # Find the index of the movie_id in all_movie_ids
    user_profile[movie_index] = rating

# Imputer les films non notés avec la note moyenne de l'utilisateur
average_rating = user_ratings['rating'].mean()
user_profile[user_profile == 0] = average_rating  

print(f"User Profile Vector for User ID {user_id}:")
print(user_profile)


User Profile Vector for User ID 1:
[3.62831858 3.62831858 3.62831858 ... 3.62831858 3.62831858 3.62831858]


In [7]:
# Maintenant, nous allons créer une collection de profils d'utilisateurs pour les besoins de cet exemple
# Pour plus de simplicité, nous allons simplement dupliquer le profil de cet utilisateur
# On aurait également pu creer des profils distincts pour des utilisateur différents
user_profiles = np.array([user_profile for _ in range(10)])  # Replace with your actual user profiles data

# On lance une instance MovieLensEnv avec une collection de user_profiles
env = MovieLensEnv(user_profiles, all_movie_ids)

# Réinitialiser l'environnement 
initial_state = env.reset()

print(f"Initial State (User Profile): {initial_state}")

action_index = np.random.choice(len(all_movie_ids))


movie_id_to_recommend = all_movie_ids[action_index]
next_state, reward, done, info = env.step(movie_id_to_recommend)

print(f"Next State (User Profile): {next_state}")
print(f"Reward: {reward}")
print(f"Done: {done}")

# Le rendu de l'environnement peut montrer le profil de l'utilisateur actuel et le film recommandé.

Initial State (User Profile): [3.62831858 3.62831858 3.62831858 ... 3.62831858 3.62831858 3.62831858]
Next State (User Profile): [3.62831858 3.62831858 3.62831858 ... 3.62831858 3.62831858 3.62831858]
Reward: 2.5
Done: False


  Etat Initial (User Profile): Il s'agit de la représentation de l'état du profil de l'utilisateur au début d'un épisode. Les chiffres qu'ont voit 3.62831858 répétés tout au long du vecteur, sont la note moyenne donnée par l'utilisateur à tous les films notés ou une note moyenne par défaut pour les films non notés. Ce vecteur sert d'entrée à notre agent RL qui l'utilisera pour décider quel film recommander.

   État suivant (Next State): Une fois que l'agent a effectué une action (recommandé un film), l'environnement passe à l'état suivant. Puisqu'on itère sur les utilisateurs en séquence, cet état suivant est le profil de l'utilisateur suivant dans notre ensemble de données. Le profil comporte les mêmes valeurs de remplacement, indiquant soit que l'utilisateur suivant a une note moyenne similaire à celle de l'utilisateur initial, soit qu'on utilise une note moyenne par défaut pour les films non notés.

  Récompense (Reward): L'agent a reçu une récompense de 2,5 pour l'action entreprise. Dans le contexte d'un système de recommandation de films, cette récompense pourrait représenter une évaluation neutre, indiquant éventuellement que le film recommandé n'a été évalué ni positivement ni négativement par l'utilisateur, ou que le film n'avait pas été évalué par l'utilisateur auparavant et qu'on utilise une valeur neutre comme récompense.

  Terminé (Done):Cette valeur booléenne indique si l'épisode est terminé. La valeur False signifie que l'épisode est toujours en cours. Dans notre configuration d'apprentissage par renforcement, un épisode peut représenter une séquence de recommandations pour un utilisateur ou un ensemble d'utilisateurs.

  Profil d'utilisateur rendu (Rendered User Profile): Il s'agit apparemment du même état que l'état initial, produit dans le cadre d'une fonction de rendu. La fonction de rendu est généralement utilisée pour visualiser ou produire l'état actuel de l'environnement sous une forme lisible par l'homme. Elle montre le profil de l'utilisateur actuel auquel l'agent a fait une recommandation.

En résumé, l'agent RL interagit avec l'environnement en recommandant des films aux utilisateurs. L'état représente les profils des utilisateurs, l'action est une recommandation de film, la récompense reflète la satisfaction de l'utilisateur et le « done » indique la fin d'une séquence d'interaction. Chaque étape franchie par l'agent lui donne des informations sur l'efficacité de ses recommandations, qui seront utilisées pour améliorer sa politique au fil du temps

In [8]:
def update_q_value(Q_table, state, action, reward, next_state, alpha, gamma, possible_actions):
    #S'assurer que l'état et l'état suivant sont des tuples afin qu'ils puissent être utilisés comme dictionary keys
    state_key = tuple(state)
    next_state_key = tuple(next_state)
    
    #Calculer la valeur maximale q-value pour les actions dans l'état suivant
    max_future_q = max(Q_table.get((next_state_key, a), 0) for a in possible_actions)  # Use a default Q-value of 0 if not found
    
    # Calculer la valeur actuelle q-value
    current_q = Q_table.get((state_key, action), 0)  # if the state-action pair is new, assume a default Q-value of 0
    
    # Calculer la nouvelle valeur q-value
    new_q = (1 - alpha) * current_q + alpha * (reward + gamma * max_future_q)
    
    # Mettre à jour Q-table avec la nouvelle valeur de q-value
    Q_table[(state_key, action)] = new_q
    return new_q  # Le renvoi de la nouvelle valeur q-value qui peut être utile à des fins de logging ou de debugging.


In [9]:
from tqdm.notebook import tqdm
# Defir les hyperparamètres
num_episodes = 3  # Le nombre d'épisodes pour lesquels la simulation doit être effectuée
learning_rate = 0.1  # Le taux d'apprentissage pour la mise à jour du  Q-learning 
discount_factor = 0.99  # Le facteur d'actualisation des récompenses futures

# Initialiser Q-table 
#Pour simplifier, il peut s'agir d'une table ou d'un réseau de neurone en fonction de la complexité.
Q_table = {}
possible_actions = list(range(len(env.movie_list)))
# Boucle d'apprentissage
for episode in tqdm(range(num_episodes)):
    state = env.reset()  # Démarrer un nouvel épisode et obtenir l'état initial
    done = False
    total_reward = 0
    
    while not done:
        #Choisir une action en fonction de la politique actuelle
        #  Pour l'instant, nous supposons une politique aléatoire
        action = np.random.choice(env.movie_list)
        
        # Prendre l'action et observer le prochain état et la prochaine récompense
        next_state, reward, done, _ = env.step(action)
        state_tuple = tuple(state)
        next_state_tuple = tuple(next_state)

        update_q_value(Q_table, state_tuple, action, reward, next_state_tuple, learning_rate, discount_factor, possible_actions)
       
        #Cumuler les récompenses
        total_reward += reward
        
        # Passage à l'état suivant
        state = next_state
    
    # Enregistrez la récompense totale pour cet épisode
    print(f"Episode {episode + 1}: Total Reward = {total_reward}")

    # En option, on peut ajouter du code pour évaluer la politique ici et interrompre la boucle plus tôt si une certaine performance est atteinte.

  0%|          | 0/3 [00:00<?, ?it/s]

Episode 1: Total Reward = 293.800986377622
Episode 2: Total Reward = 159.6557247529296
Episode 3: Total Reward = 1098.529108361166


 p.s. 

La Q-table est une matrice dans laquelle il y a une ligne pour chaque état et une colonne pour chaque action. Elle est d'abord initialisée à 0 puis les valeurs sont mises à jour après l'entraînement. La Q-table a les mêmes dimensions que la reward table, mais elle a un objectif complètement différent.

In [ ]:
#import pickle
#  Après l'entrainement,on peut enregistrer notre modèle ou notre Q-table pour un usage ultérieur.
#with open('q_table.pickle', 'wb') as handle:
    #pickle.dump(Q_table, handle, protocol=pickle.HIGHEST_PROTOCOL)

En deep Q-learning, on remplace la Q-table par un réseau neuronal qui approxime les valeurs Q-values  pour chaque paire état-action..

Voici une vue d'ensemble de ce que nous allons faire :

Définir une architecture de réseau neuronal Q-network.
Remplacer la recherche et la mise à jour dans la Q-table par des passages avant et arrière dans le réseau neuronal.
Implementer e renouvellement de l'expérience afin d'échantillonner de manière aléatoire les expériences pour l'entrainement du  Q-network.
Utiliser un réseau cible pour stabiliser l'entrainement,qui est périodiquement mis à jour avec les poids du Q-network.

In [10]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Définir l'architecture Q-Network 
class QNetwork(nn.Module):
    def __init__(self, state_size, action_size, hidden_size=64):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, action_size)
        
    def forward(self, state):
        x = self.fc1(state)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Initialiser le Q-Network et le réseau cible
state_size = len(state)  # TLa taille de la représentation de l'État
action_size = len(possible_actions)  # Le nombre d'actions possibles
q_network = QNetwork(state_size, action_size)
target_network = QNetwork(state_size, action_size)
target_network.load_state_dict(q_network.state_dict())

optimizer = optim.Adam(q_network.parameters(), lr=0.1)
criterion = nn.MSELoss()

# La fonction de mise à jour prend désormais en compte un ensemble d'expériences
def update_q_network(batch, gamma, alpha):
    states, actions, rewards, next_states, dones = zip(*batch)
    
    states = torch.FloatTensor(states)
    actions = torch.LongTensor(actions)
    rewards = torch.FloatTensor(rewards)
    next_states = torch.FloatTensor(next_states)
    dones = torch.ByteTensor(dones)

    # Obtenir les Q values pour les états actuels
    Q_values = q_network(states).gather(1, actions.unsqueeze(1)).squeeze(1)
    
    #Calculer les  Q values pour les états suivants en utilisant le réseau cible
    Q_next = target_network(next_states).detach()
    Q_next_values = Q_next.max(1)[0]
    Q_targets = rewards + gamma * Q_next_values * (1 - dones)
    
    #  Calculer la perte
    loss = criterion(Q_values, Q_targets)
    
    #  Optimiser le modèle
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Optionnellement, mettre à jour le réseau cible avec les poids du  Q-network's
    # Cela pourrait se faire à intervalles réguliers, pas nécessairement à chaque mise à jour.
    target_network.load_state_dict(q_network.state_dict())

# Experience replay buffer
class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = []
        self.position = 0
    
    def push(self, state, action, reward, next_state, done):
        if len(self.buffer) < self.capacity:
            self.buffer.append(None)
        self.buffer[self.position] = (state, action, reward, next_state, done)
        self.position = (self.position + 1) % self.capacity
    
    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)
    
    def __len__(self):
        return len(self.buffer)

# Exemple d'usage
replay_buffer = ReplayBuffer(10000)
batch_size = 32

#  Supposons que nous disposions d'un moyen d'obtenir des expériences et de les stocker dans replay buffer
# Pour state, action, reward, next_state, done dans les expériences:
#     replay_buffer.push(state, action, reward, next_state, done)

# Entraînement du réseau
if len(replay_buffer) > batch_size:
    batch = replay_buffer.sample(batch_size)
    update_q_network(batch, gamma, alpha)
